In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("linear_regression").getOrCreate()

In [11]:
df=spark.read.csv('Ecommerce_Customers.csv',header=True,inferSchema=True)

In [12]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [8]:
df.columns 

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
df.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length='34.49726772511229', Time on App='12.65565114916675', Time on Website='39.57766801952616', Length of Membership='4.0826206329529615', Yearly Amount Spent='587.9510539684005')]

### Features vs Labels Encoding

In [14]:
from pyspark.ml.feature import VectorAssembler

In [21]:
assembler=VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [22]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [23]:
output=assembler.transform(df)

In [26]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [25]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [27]:
final_data=output.select(['features','Yearly Amount Spent'])

In [28]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [29]:
final_data.head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]), Yearly Amount Spent=587.9510539684005)]

In [30]:
final_data.head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]), Yearly Amount Spent=587.9510539684005)]

## Train-Test Splits

In [31]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [32]:
train_data.head(1)

[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464]), Yearly Amount Spent=408.6403510726275)]

In [33]:
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.3931845423455...|  319.9288698031936|
|[30.4925366965402...|  282.4712457199145|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8162006488763...|   266.086340948469|
|[30.8794843441274...|  490.2065999848547|
|[30.9716756438877...|  494.6386097568927|
|[31.0472221394875...|  392.4973991890214|
|[31.0613251567161...|  487.5554580579016|
|[31.0662181616375...| 448.93329320767435|
|[31.1239743499119...|  486.9470538397658|
|[31.1280900496166...|  557.2526867470547|
|[31.2606468698795...|  421.3266312569514|
|[31.2681042107507...|  423.4705331738239|
|[31.2834474760581...|  591.7810894256675|
|[31.3091926408918...|  432.7207178399336|
|[31.3584771924370...|  495.1759504494754|
|[31.3662121671876...|  430.5888825564849|
|[31.3895854806643...|  410.0696110599829|
+----------

In [34]:
test_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.8364326747734...|  467.5019004269896|
|[31.1695067987115...|  427.3565308022928|
|[31.3123495994443...|  463.5914180279406|
|[31.4459724827577...| 484.87696493512857|
|[31.4474464941278...|   418.602742095224|
|[31.5171218025062...|  275.9184206503857|
|[31.5702008293202...|  545.9454921414049|
|[31.7216523605090...| 347.77692663187264|
|[31.7656188210424...| 496.55408163560713|
|[31.8093003166791...|  536.7718993628412|
|[31.8124825597242...|  392.8103449837972|
|[31.8164283341993...| 501.12249150365636|
|[31.8512531286083...|  472.9922466667984|
|[31.8648325480987...|  439.8912804768137|
|[31.9048571310136...| 473.94985742281614|
|[31.9262720263601...|  392.2049334443264|
|[31.9365486184489...|  427.1993848953282|
|[31.9480174211613...|  461.9208768928978|
|[31.9549038566348...|   439.997879939927|
|[31.9673209478824...| 445.74984123965226|
+----------

### Machine Learning

In [35]:
from pyspark.ml.regression import LinearRegression

In [36]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [37]:
lr_model=lr.fit(train_data)

### Model Evaluation

In [39]:
test_results=lr_model.evaluate(test_data)

In [40]:
test_results.meanAbsoluteError

8.468949449868775

In [41]:
test_results.meanSquaredError

107.49311687891402

In [42]:
test_results.rootMeanSquaredError

10.36788873777656

### Making Predictions

In [43]:
data=test_data.select(['features'])

In [44]:
pred_data=lr_model.transform(data)

In [45]:
pred_data.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...| 471.4304550614213|
|[31.1695067987115...| 416.6490555867258|
|[31.3123495994443...|   443.67824956036|
|[31.4459724827577...| 481.1641389992178|
|[31.4474464941278...| 425.6826033099419|
|[31.5171218025062...|279.46341614376524|
|[31.5702008293202...| 563.1878312841498|
|[31.7216523605090...|349.31882031450846|
|[31.7656188210424...| 500.3448114017722|
|[31.8093003166791...| 547.8955797075098|
|[31.8124825597242...| 396.8967770849729|
|[31.8164283341993...| 518.2324795953277|
|[31.8512531286083...| 464.6538557707281|
|[31.8648325480987...|449.77624014073285|
|[31.9048571310136...|490.72034031798376|
|[31.9262720263601...| 380.0066781607229|
|[31.9365486184489...|440.60339784506345|
|[31.9480174211613...| 455.8642239604512|
|[31.9549038566348...|431.39386374080095|
|[31.9673209478824...| 449.0171859469997|
+--------------------+------------